### Step 01: Add Imports

In [1]:
import os

In [2]:
import dash

In [3]:
from dash import dcc

In [4]:
from dash import html

In [5]:
from dash.dependencies import Input, Output

In [6]:
from keras.models import load_model

In [7]:
import joblib

In [8]:
import numpy as np

In [9]:
import pandas as pd

In [10]:
from sklearn.preprocessing import StandardScaler

### Step 02: Load the Model

In [11]:
model = load_model('diamond-neural-network.h5')

In [12]:
pca = joblib.load('pca.joblib')

/Users/josevila/anaconda3/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 0.23.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### Step 03: Create the HTML and use the exteranl CSS file:

In [13]:
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

In [14]:
input_carat = dcc.Input(id='carat', type='numeric', value=0.7)

In [15]:
div_carat = html.Div(children=[html.H3('Carat:'), input_carat], className='four columns')

In [16]:
input_depth = dcc.Input(id='depth', placeholder='', type='numeric', value=60)

In [17]:
div_depth = html.Div(children=[html.H3('Depth:'), input_depth], className='four columns')

In [18]:
input_table = dcc.Input(id='table', placeholder='', type='numeric', value=60)

In [19]:
div_table = html.Div(children=[html.H3('Table:'), input_table], className='four columns')

In [20]:
input_x = dcc.Input(id='x', placeholder='', type='numeric', value=5)

In [21]:
div_x = html.Div(children=[html.H3('x value:'), input_x], className='four columns')

In [22]:
input_y = dcc.Input(id='y', placeholder='', type='numeric', value=5)

In [23]:
div_y = html.Div(children=[html.H3('y value:'), input_y], className='four columns')

In [24]:
input_z = dcc.Input(id='z', placeholder='', type='numeric', value=3)

In [25]:
div_z = html.Div(children=[html.H3('z value:'), input_z], className='four columns')

In [26]:
cut_values = ['Fair', 'Good', 'Ideal', 'Premium', 'Very Good']

In [27]:
cut_options = [ {'label': x, 'value': x} for x in cut_values ]

In [28]:
input_cut = dcc.Dropdown(id='cut', options = cut_options, value = 'Ideal')

In [29]:
div_cut = html.Div(children=[html.H3('Cut:'), input_cut], className='four columns')

In [30]:
color_values = ['D', 'E', 'F', 'G', 'H', 'I', 'J']

In [31]:
color_options = [ {'label': x, 'value': x} for x in color_values ]

In [32]:
input_color = dcc.Dropdown(id='color', options = color_options, value = 'G')

In [33]:
div_color = html.Div(children=[html.H3('Color:'), input_color], className='four columns')

In [34]:
clarity_values = ['I1', 'IF', 'SI1', 'VS1', 'VS2', 'VVS1', 'VVS2']

In [35]:
clarity_options = [ {'label': x, 'value': x} for x in clarity_values ]

In [36]:
input_clarity = dcc.Dropdown(id='clarity', options = clarity_options, value = 'SI1')

In [37]:
div_clarity = html.Div(children=[html.H3('Clarity:'), input_clarity], className='four columns')

In [38]:
div_numerical = html.Div(children=[div_carat, div_depth, div_table], className='row')

In [39]:
div_dimensions = html.Div(children=[div_x, div_y, div_z], className='row')

In [40]:
div_categorical = html.Div(children=[div_cut, div_color, div_clarity], className='row')

### Step 04: Use the model with the inputs from Dash:

In [41]:
def get_prediction(carat, depth, table, x, y, z, cut, color, clarity):
    cols = ['carat', 'depth', 'table', 'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 'dim_index']

    cut_dict = { x: 'cut_' + x for x in cut_values[1:] }
    color_dict = { x: 'color_' + x for x in color_values[1:] }
    clarity_dict = { x: 'clarity_' + x for x in clarity_values[1:] }

    df = pd.DataFrame(data=np.zeros( (1, len(cols)) ), columns=cols)
    df.loc[0, 'carat'] = carat
    df.loc[0, 'depth'] = depth
    df.loc[0, 'table'] = table

    dims_df = pd.DataFrame(data=[[x, y, z]], columns=['x', 'y', 'z'])
    df.loc[0, 'dim_index'] = pca.transform(dims_df).flatten()[0]

    if cut != 'Fair':
        df.loc[0, cut_dict[cut]] = 1
    
    if color != 'D':
        df.loc[0, color_dict[color]] = 1

    if clarity != 'I1':
        df.loc[0, clarity_dict[clarity]] = 1
    
    scaler = StandardScaler()
    
    numerical_features = ['carat', 'depth', 'table', 'dim_index']
    
    df.loc[:, numerical_features] = scaler.fit_transform(df.loc[:, numerical_features])

    prediction = model.predict(df.values).flatten()[0]
    
    #prediction = np.exp(prediction)

    return int(prediction)

In [42]:
app.layout = html.Div([html.H1('IDR Predict diamond prices'), html.H2('Enter the diamond characteristics to get the predicted price'), html.Div(children=[div_numerical, div_dimensions, div_categorical]), html.H1(id='output', style = {'margin-top': '50px', 'text-align': 'center'})])

In [43]:
predictors = ['carat', 'depth', 'table', 'x', 'y', 'z', 'cut', 'color', 'clarity']

In [44]:
### Step 05: Display the Predicted Price

In [45]:
@app.callback(Output('output', 'children'), [Input(x, 'value') for x in predictors])
def show_prediction(carat, depth, table, x, y, z, cut, color, clarity):
    pred = get_prediction(carat, depth, table, x, y, z, cut, color, clarity)
    return str('Predicted Price: {:,}'.format(pred))

In [46]:
if __name__ == '__main__':
    app.run_server(debug=False)

1/1 [==============================] - 0s 34ms/step


/Users/josevila/anaconda3/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning:

X has feature names, but PCA was fitted without feature names

